In [1]:
import numpy as np
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from tqdm import tqdm

In [2]:
queryString = "SELECT * WHERE { ?s ?p ?o. }"
sparql = SPARQLWrapper("http://localhost:3030/MergeAll/query")
sparql.setQuery(queryString)

try :
   ret = sparql.query()
   # ret is a stream with the results in XML, see <http://www.w3.org/TR/rdf-sparql-XMLres/>
except :
   pass

In [3]:
sparql.setQuery("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?s (COUNT(?prop) AS ?total) {
  
  SELECT DISTINCT ?s ?prop
    WHERE {
    ?s wdt:P31 wd:Q515 .
    ?s ?prop ?value .
} 

} GROUP BY ?s
ORDER BY DESC(?total) 
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [4]:
res = []
for results in results["results"]["bindings"]:
    print('%s: %s' % (results["s"]["value"], results["total"]["value"]))
    entity = str(results["s"]["value"]).split('/')
    res.append(entity[-1])
print('---------------------------')

http://www.wikidata.org/entity/Q334: 155
http://www.wikidata.org/entity/Q8646: 144
http://www.wikidata.org/entity/Q60: 111
http://www.wikidata.org/entity/Q220: 99
http://www.wikidata.org/entity/Q2096: 97
http://www.wikidata.org/entity/Q1726: 96
http://www.wikidata.org/entity/Q2807: 96
http://www.wikidata.org/entity/Q1492: 95
http://www.wikidata.org/entity/Q62: 95
http://www.wikidata.org/entity/Q1218: 94
http://www.wikidata.org/entity/Q727: 93
http://www.wikidata.org/entity/Q956: 93
http://www.wikidata.org/entity/Q1486: 92
http://www.wikidata.org/entity/Q84: 92
http://www.wikidata.org/entity/Q172: 90
http://www.wikidata.org/entity/Q2044: 89
http://www.wikidata.org/entity/Q490: 89
http://www.wikidata.org/entity/Q1731: 88
http://www.wikidata.org/entity/Q641: 88
http://www.wikidata.org/entity/Q12258: 87
http://www.wikidata.org/entity/Q239: 87
http://www.wikidata.org/entity/Q2634: 87
http://www.wikidata.org/entity/Q3992: 87
http://www.wikidata.org/entity/Q8810: 87
http://www.wikidata.org/en

http://www.wikidata.org/entity/Q1906266: 20
http://www.wikidata.org/entity/Q1913327: 20
http://www.wikidata.org/entity/Q1922330: 20
http://www.wikidata.org/entity/Q19258: 20
http://www.wikidata.org/entity/Q1937588: 20
http://www.wikidata.org/entity/Q1946027: 20
http://www.wikidata.org/entity/Q1946056: 20
http://www.wikidata.org/entity/Q1946077: 20
http://www.wikidata.org/entity/Q1949422: 20
http://www.wikidata.org/entity/Q1994818: 20
http://www.wikidata.org/entity/Q2000579: 20
http://www.wikidata.org/entity/Q202889: 20
http://www.wikidata.org/entity/Q205792: 20
http://www.wikidata.org/entity/Q206125: 20
http://www.wikidata.org/entity/Q2061265: 20
http://www.wikidata.org/entity/Q2090598: 20
http://www.wikidata.org/entity/Q2096693: 20
http://www.wikidata.org/entity/Q2158248: 20
http://www.wikidata.org/entity/Q217426: 20
http://www.wikidata.org/entity/Q219093: 20
http://www.wikidata.org/entity/Q219417: 20
http://www.wikidata.org/entity/Q2249428: 20
http://www.wikidata.org/entity/Q2269545:

http://www.wikidata.org/entity/Q33432813: 8
http://www.wikidata.org/entity/Q33433503: 8
http://www.wikidata.org/entity/Q33435346: 8
http://www.wikidata.org/entity/Q33528284: 8
http://www.wikidata.org/entity/Q3368418: 8
http://www.wikidata.org/entity/Q3408680: 8
http://www.wikidata.org/entity/Q3412425: 8
http://www.wikidata.org/entity/Q3412434: 8
http://www.wikidata.org/entity/Q3442746: 8
http://www.wikidata.org/entity/Q3449469: 8
http://www.wikidata.org/entity/Q3461465: 8
http://www.wikidata.org/entity/Q3464234: 8
http://www.wikidata.org/entity/Q3469700: 8
http://www.wikidata.org/entity/Q3472184: 8
http://www.wikidata.org/entity/Q3476942: 8
http://www.wikidata.org/entity/Q3478042: 8
http://www.wikidata.org/entity/Q3485061: 8
http://www.wikidata.org/entity/Q3494688: 8
http://www.wikidata.org/entity/Q3523969: 8
http://www.wikidata.org/entity/Q3524626: 8
http://www.wikidata.org/entity/Q3559224: 8
http://www.wikidata.org/entity/Q3563897: 8
http://www.wikidata.org/entity/Q3651040: 8
http://

In [5]:
db = []

for i in range(len(res)):
    query_string = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    SELECT DISTINCT ?s ?prop {
    VALUES ?s {wd:""" + res[i] + """}
    ?s ?prop ?value .
    }
    """

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results_entity = sparql.query().convert()
    propLabel = []
    for results in results_entity["results"]["bindings"]:
#         print('%s: %s' % (results["country"]["value"], results["propLabel"]["value"]))
        propLabel.append(results["prop"]["value"])
#     print('---------------------------')
    db.append(propLabel)

In [6]:
te = TransactionEncoder()
te_ary = te.fit(db).transform(db)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P1001,http://www.wikidata.org/prop/P1012,http://www.wikidata.org/prop/P1019,http://www.wikidata.org/prop/P1028,http://www.wikidata.org/prop/P103,http://www.wikidata.org/prop/P1056,http://www.wikidata.org/prop/P106,http://www.wikidata.org/prop/P1081,http://www.wikidata.org/prop/P1082,...,http://www.wikidata.org/prop/direct/P868,http://www.wikidata.org/prop/direct/P8744,http://www.wikidata.org/prop/direct/P88,http://www.wikidata.org/prop/direct/P8933,http://www.wikidata.org/prop/direct/P8989,http://www.wikidata.org/prop/direct/P910,http://www.wikidata.org/prop/direct/P912,http://www.wikidata.org/prop/direct/P92,http://www.wikidata.org/prop/direct/P9241,http://www.wikidata.org/prop/direct/P971
0,False,False,False,False,False,False,False,False,True,True,...,False,True,False,False,False,True,False,False,True,False
1,False,False,False,False,False,False,False,False,True,True,...,False,True,False,True,True,True,False,True,True,False
2,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,True,True,False,False,True,False
3,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,True,False
4,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8919,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8920,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8921,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8922,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
wikidata = SPARQLWrapper("https://query.wikidata.org/sparql")

In [8]:
propList = df.columns.tolist()
for i in range(len(propList)):
    propList[i]=propList[i].split('/')[-1]

In [9]:
propLabel = []

for i in tqdm(range(len(propList))):
    query_string = """
    SELECT DISTINCT ?propLabel {
      VALUES ?p {wdt:""" + propList[i] + """}
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
      ?prop wikibase:directClaim ?p .
    }
    """

    wikidata.setQuery(query_string)
    wikidata.setReturnFormat(JSON)
    results_prop = wikidata.query().convert()
    for results in results_prop["results"]["bindings"]:
#         print('%s: %s' % (results["country"]["value"], results["propLabel"]["value"]))
        propLabel.append(results["propLabel"]["value"])
#     print('---------------------------')

100%|████████████████████████████████████████████████████████████████████████████████| 491/491 [06:05<00:00,  1.34it/s]


In [10]:
df.columns = propLabel
df

,video,applies to jurisdiction,including,web feed URL,donated by,native language,product or material produced,occupation,Human Development Index,population,...,foods traditionally associated,economy of topic,commissioned by,category for the view from the item,category for the view of the item,topic's main category,has facility,main regulatory text,demographics of topic,category combines topics
0,False,False,False,False,False,False,False,False,True,True,...,False,True,False,False,False,True,False,False,True,False
1,False,False,False,False,False,False,False,False,True,True,...,False,True,False,True,True,True,False,True,True,False
2,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,True,True,False,False,True,False
3,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,True,False
4,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8919,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8920,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8921,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8922,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
from mlxtend.frequent_patterns import association_rules, fpmax, fpgrowth
frequent_itemsets = fpgrowth(df, min_support=0.5, use_colnames=True)
frequent_itemsets

,support,itemsets
0,1.000000,(instance of)
1,1.000000,(instance of)
2,0.946997,(country)
3,0.946773,(country)
4,0.857575,(coordinate location)
...,...,...
634,0.786082,"(instance of, located in the administrative te..."
635,0.786082,"(instance of, located in the administrative te..."
636,0.786082,"(instance of, located in the administrative te..."
637,0.786082,"(instance of, located in the administrative te..."


In [12]:
res = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(instance of),(country),1.000000,0.946773,0.946773,0.946773,1.000000,0.000000,1.000000
1,(country),(instance of),0.946773,1.000000,0.946773,1.000000,1.000000,0.000000,inf
2,(instance of),(coordinate location),1.000000,0.857575,0.857575,0.857575,1.000000,0.000000,1.000000
3,(coordinate location),(instance of),0.857575,1.000000,0.857575,1.000000,1.000000,0.000000,inf
4,(country),(coordinate location),0.946773,0.857575,0.852308,0.900225,1.049733,0.040380,1.427459
...,...,...,...,...,...,...,...,...,...
485,"(country, coordinate location)","(instance of, located in the administrative te...",0.852308,0.843904,0.786082,0.922298,1.092895,0.066816,2.008910
486,(instance of),(located in the administrative territorial ent...,1.000000,0.786082,0.786082,0.786082,1.000000,0.000000,1.000000
487,(located in the administrative territorial ent...,"(instance of, country, coordinate location)",0.843904,0.852308,0.786082,0.931483,1.092895,0.066816,2.155553
488,(country),"(instance of, located in the administrative te...",0.946773,0.786867,0.786082,0.830276,1.055167,0.041098,1.255761
